In [1]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split

In [2]:
data_files = glob.glob('data/*.csv')

In [10]:
data = []
for file in data_files:
    data.append(pd.read_csv(file))
    
data = pd.concat(data, axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (33,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
data.columns.values

array(['CLAV_CA_HI_ID', 'CLAV_CA_NK_ID', 'COMPANY_CODE', 'CLAIM_BR',
       'CLAIM_NO', 'SEQ_NO__CA', 'ABBR_NAME', 'POLICY_BR', 'POLICY_NO',
       'POLICY_TYP', 'CLIENT_BR', 'CLIENT_NO', 'ACCOUNT_BR', 'ACCOUNT_NO',
       'TERM_ID', 'TRAN_DATE', 'TRAN_TIME', 'TRAN_USER', 'VALID_FLAG',
       'RISK_NO', 'RI_METHOD', 'RI_TRNO', 'DATE_REPT', 'DATE_OCC',
       'DATE_INC', 'DATE_EXP', 'RSK_CLASS', 'CLAIMANT', 'CLM_INITIAL',
       'CLM_DESC1', 'HS_FLAG', 'SOLICTOR_A', 'SOLICTOR_B', 'CO_CAT',
       'ICA_CAT', 'SUBROG', 'ASSESSR_A', 'ASSESSR_B', 'CL_INIT',
       'ESTIMATE', 'CHANGE_DATE', 'CLAIM_STATUS', 'REOPEN', 'PAY_BLOCK',
       'ACCPT_FLG', 'BAL_OUTS', 'PAID', 'INCURRED', 'BO_TU', 'PAID_TU',
       'INC_TU', 'OTHER_PARTY', 'SUB_CLASS', 'T_PARTY', 'STATUS_DATE',
       'RATE_NCB', 'PREINC', 'RECOVERY', 'SETTLING_BR', 'RECOVERY_TU',
       'ACTIVITY_DATE', 'DENIED_FLAG', 'REASON_DENIED', 'INVESTIGATOR_A',
       'INVESTIGATOR_B', 'OLD_ARCHIVE_BOX', 'DESTROY_DATE', 'ARCHIVE_BOX',
     

In [16]:
strat = data.RECOVERY_FLAG

In [24]:
_, data_small = train_test_split(data, test_size=0.1, stratify=strat)

In [25]:
data_small.shape

(55797, 85)

In [31]:
data_small.to_csv('sample_data.csv', index=False)

In [27]:
del data

In [2]:
data_small = pd.read_csv('sample_data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (33,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
[col for col in data_small.columns if col.endswith('DESC1')]

['CLM_DESC1']

In [30]:
data_small.columns.values

array(['CLAV_CA_HI_ID', 'CLAV_CA_NK_ID', 'COMPANY_CODE', 'CLAIM_BR',
       'CLAIM_NO', 'SEQ_NO__CA', 'ABBR_NAME', 'POLICY_BR', 'POLICY_NO',
       'POLICY_TYP', 'CLIENT_BR', 'CLIENT_NO', 'ACCOUNT_BR', 'ACCOUNT_NO',
       'TERM_ID', 'TRAN_DATE', 'TRAN_TIME', 'TRAN_USER', 'VALID_FLAG',
       'RISK_NO', 'RI_METHOD', 'RI_TRNO', 'DATE_REPT', 'DATE_OCC',
       'DATE_INC', 'DATE_EXP', 'RSK_CLASS', 'CLAIMANT', 'CLM_INITIAL',
       'CLM_DESC1', 'HS_FLAG', 'SOLICTOR_A', 'SOLICTOR_B', 'CO_CAT',
       'ICA_CAT', 'SUBROG', 'ASSESSR_A', 'ASSESSR_B', 'CL_INIT',
       'ESTIMATE', 'CHANGE_DATE', 'CLAIM_STATUS', 'REOPEN', 'PAY_BLOCK',
       'ACCPT_FLG', 'BAL_OUTS', 'PAID', 'INCURRED', 'BO_TU', 'PAID_TU',
       'INC_TU', 'OTHER_PARTY', 'SUB_CLASS', 'T_PARTY', 'STATUS_DATE',
       'RATE_NCB', 'PREINC', 'RECOVERY', 'SETTLING_BR', 'RECOVERY_TU',
       'ACTIVITY_DATE', 'DENIED_FLAG', 'REASON_DENIED', 'INVESTIGATOR_A',
       'INVESTIGATOR_B', 'OLD_ARCHIVE_BOX', 'DESTROY_DATE', 'ARCHIVE_BOX',
     

In [4]:
data_small.INCIDENT_DESC.fillna('')
data_small.INCIDENT_DESC = data_small.INCIDENT_DESC.astype(str)
data_small.CLM_DESC1 = data_small.CLM_DESC1.astype(str)
data_small['text'] = data_small.apply(lambda x: x.CLM_DESC1 if len(x.INCIDENT_DESC) < 7 else x.INCIDENT_DESC, axis=1)

In [44]:
data_small.text[:5]

241760                                          INSD HIT TP
302326                        TP HIT INSURED PARKED VEHICLE
46352     THIRD PARTY HAS REVERSED INTO INSURED AND THEN...
242319                                  TP HIT INSD IN REAR
43697     SUICIDAL KANGAROO JUMPED OUT FROM BEHIND A LAR...
Name: text, dtype: object

In [5]:
unstr_data = data_small[['text', 'RECOVERY_FLAG']]

In [6]:
unstr_data.columns = [col.lower() for col in unstr_data.columns]

In [7]:
unstr_data[:5]

,text,recovery_flag
0,INSD HIT TP,0
1,TP HIT INSURED PARKED VEHICLE,0
2,THIRD PARTY HAS REVERSED INTO INSURED AND THEN...,0
3,TP HIT INSD IN REAR,1
4,SUICIDAL KANGAROO JUMPED OUT FROM BEHIND A LAR...,0


In [14]:
import string
import re
from nltk import word_tokenize
from nltk.corpus import stopwords

stop = set(stopwords.words('english')).union(set(string.punctuation))

def unescape(s):
    '''This is a function to remove stubborn garbage in the text'''

    s = str(s)
    s = re.sub(r"<.*?>", "", s)
    s = s.replace("&lt;", "less").replace("&gt;", "greater").replace("&amp;", "and").replace("x0D", "")
    s = s.replace("\n", " ").replace("\r", " ").replace("*", " ").replace("&#;", '').replace("\t", "")
    s = s.replace("(", "").replace(")", "").replace("null,", "").replace('\'', "")
    s = re.sub('[^A-Za-z,.[space]]+', '', s)
    s = re.sub('[ \t]+', ' ', s)
    return s


def tokenize(txt):
    return [unescape(w) for w in word_tokenize(txt.lower()) if w not in stop and len(w) > 3]

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(norm='l2', min_df=2, use_idf=True, ngram_range=(1, 3),
                             smooth_idf=False, tokenizer=tokenize,
                             stop_words='english', max_features=500)

text = unstr_data.text
tf_idf = vectorizer.fit_transform(text)

In [19]:
type(tf_idf)

scipy.sparse.csr.csr_matrix

In [21]:
X = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names())

In [23]:
X.head()

,accident,accidentally,accidently,advised,animal,approaching,area,attached,attempted,attempting,...,windscreen breakage,windscreen claim,windscreen cracked,windscreen damage,windscreen damaged,windscreen window,windscreen window damage,work,wrong,yard
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                             min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,
                             bootstrap=True, oob_score=False, n_jobs=1,
                             random_state=None, verbose=0, warm_start=False, class_weight='balanced')

In [28]:
y = unstr_data.recovery_flag

In [30]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
clf.score(X, y)